In [2]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import data
import pandas as pd


class Window(tk.Tk):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.title("信用卡消費樣態")

        #-----------interface-----------#

        #------搜尋------#
        topFrame = ttk.Labelframe(self, text="搜尋")
        #-------Label------#
        self.dataLabel = ttk.Label(topFrame, text="資料類別:").grid(row=0, column=0,padx=10, pady=10)
        self.yearLabel = ttk.Label(topFrame, text='年份:').grid(row=0, column=2, padx=10, pady=10)
        self.monthLabel = ttk.Label(topFrame, text='月份:').grid(row=0, column=4, padx=10, pady=10)
        self.industryLabel = ttk.Label(topFrame, text="產業別:").grid(row=0, column=6, padx=10, pady=10)
        #------StringVar------#
        self.data_var = tk.StringVar()
        self.data_var.set("Select Data")
        self.data = ttk.Combobox(topFrame,textvariable=self.data_var,values=["職業", "年收入", "教育程度", "兩性", "年齡"])
        self.data.grid(row=0, column=1, padx=10, pady=10)
        self.data.bind("<<ComboboxSelected>>", self.load_data)

        self.year_var = tk.StringVar()
        self.year_var.set("Select Year")
        self.year = ttk.Combobox(topFrame,textvariable=self.year_var,values=["2014","2015","2016","2017","2018","2019","2020","2021","2022","2023"])
        self.year.grid(row=0, column=3, padx=10, pady=10)
        self.year.bind("<<ComboboxSelected>>", self.load_year)

        self.month_var = tk.StringVar()
        self.month_var.set("Select Month")
        self.month = ttk.Combobox(topFrame, textvariable=self.month_var, values=['一月', '二月', '三月', '四月', '五月', '六月', '七月', '八月', '九月', '十月', '十一月', '十二月'])
        self.month.grid(row=0, column=5, padx=10, pady=10)
        self.month.bind("<<ComboboxSelected>>", self.load_month)

        self.industry_var = tk.StringVar()
        self.industry_var.set("Select Industry")
        self.industry = ttk.Combobox(topFrame, textvariable=self.industry_var, values=['食', '衣', '住', '行', '文教康樂', '百貨', '其他'])
        self.industry.grid(row=0, column=7, padx=10, pady=10)
        self.industry.bind("<<ComboboxSelected>>", self.load_industry)

        topFrame.pack()
        #------------資料呈現------------#
        middleFrame = ttk.Labelframe(self, text="資料")
        self.treeview = ttk.Treeview(middleFrame, show="headings", height=20)
        self.treeview.grid(row=1, column=0, padx=10, pady=10)

        #-----垂直滾動條------#
        scrollBar = ttk.Scrollbar(
            middleFrame, orient="vertical", command=self.treeview.yview
        )
        scrollBar.grid(row=1, column=1, sticky="ns")  # 使用 grid 進行佈局管理

        self.treeview.configure(yscrollcommand=scrollBar.set)

        middleFrame.pack()

        #------------分析------------#
        bottomFrame1 = ttk.Labelframe(self, text="圖1")
        tk.Label(bottomFrame1, text="").pack()
        bottomFrame1.pack()

        bottomFrame2 = ttk.Labelframe(self, text="圖2")
        tk.Label(bottomFrame2, text="").pack()
        bottomFrame2.pack()

        bottomFrame3 = ttk.Labelframe(self, text="圖3")
        tk.Label(bottomFrame3, text="").pack()
        bottomFrame3.pack()

    def load_data(self, event):
        selected_data = self.data_var.get()
        print(f'Data:{selected_data}')
    def load_year(self, event):
        selected_year = self.year_var.get()
        print(f'Year:{selected_year}')
    def load_month(self, event):
        selected_month = self.year_var.get()
        print(f'Month:{selected_month}')
    def load_industry(self, event):
        selected_industry = self.industry_var.get()
        print(f'Industry:{selected_industry}')

def main():
    window = Window()
    window.mainloop()


if __name__ == "__main__":
    main()

Data:職業
Year:2017
Month:2017
